# Capstone<br>
## Battle of the Neighbourhoods <br>
### Where to stay in NYC<br>
Samuel Cheng

_______________________________________________________________

#### 1) Discussion and Background of the Business Problem:

**Problem Statement: Where to stay in New York City (NYC) ** 

New york city is the most populous city in United States and one of the most exciting cities in the world to live and work in. Being a transient city where students and professionals come from all around the world to live in. In choosing a place to rent in New York, users will often consider the ammenities of the neighbourhood. With this report, the aim is to highlight neighbourhoods that has good concentration of ammenities surrounding them. <br>

Target Audience: <br>
1)	Individuals looking to rent a place in New York that is surrounded by ammenities. <br>

<br>

#### 2) Methodology and Data required

Methodology:<br>
We will first obtain the neighbourhoods within New York City and their coordinates and plot it on the map to have a good appreciation of how the neighbourhoods are scattered.<br>
Thereafter, we will use four square to retrieve ammentities surrounding these neighbourhoods. <br>
As the intention of this report is to identify neighbourhoods with high concentration of ammenities, we will only select the top 50 neighbourhoods throughout new york in terms of ammenities concentration for further analysis. <br>
We will then cluster these neighbourhooods and idenitfy their common characteristics. <br>
Target audience will be able to select which clucster makes more sense to them and help them streamline their search for accomodation in New York.<br>

Data Source: <br>
a.	The borough, neighbourhoods and location data of new york city https://cocl.us/new_york_dataset <br>
b.	Using four square location data to identify the surrounding ammenities <br>





In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip -q install folium
import folium  # map rendering library

print('Libraries imported.')

Libraries imported.


#### Download and Explore Dataset

In [29]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [30]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [31]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [45]:
neighborhoods_data = newyork_data['features']

In [46]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

In [34]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


In [35]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [36]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [37]:
address = 'New York City, NY'

import pandas as pd
import requests
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [38]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##### Using Four Square Location Data

In [62]:
CLIENT_ID = 'VFJBMTZRUJWJUETLOF1ZOKT54RGUTCTOCZSR3JCBRSCG1JEK' # your Foursquare ID
CLIENT_SECRET = '4TVUF2ITDPWVCM0THUKPBSETS5SORFHMFTXPOCJQOK0X454R' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: VFJBMTZRUJWJUETLOF1ZOKT54RGUTCTOCZSR3JCBRSCG1JEK
CLIENT_SECRET:4TVUF2ITDPWVCM0THUKPBSETS5SORFHMFTXPOCJQOK0X454R


In [63]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [64]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=VFJBMTZRUJWJUETLOF1ZOKT54RGUTCTOCZSR3JCBRSCG1JEK&client_secret=4TVUF2ITDPWVCM0THUKPBSETS5SORFHMFTXPOCJQOK0X454R&v=20180604&ll=40.89470517661,-73.84720052054902&radius=500&limit=100'

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5edb469db1cac0001b12f08d'},
 'response': {}}

In [67]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [68]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

In [70]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
nyc_venues = getNearbyVenues(names= neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield


KeyError: 'groups'

In [59]:
print(nyc_venues.shape)
nyc_venues.head()

NameError: name 'nyc_venues' is not defined

In [51]:
nyc_venues_count = nyc_venues.groupby('Neighbourhood', as_index=False).count()
nyc_venues_count.head()

NameError: name 'nyc_venues' is not defined

In [52]:
nyc_venues_sorted = nyc_venues_count.sort_values(by='Venue', ascending=False)
nyc_venues_sorted

NameError: name 'nyc_venues_count' is not defined

#### Select the top 50 Neighbourhoods by concentration

In [58]:
nyc_venues_top = nyc_venues_sorted.iloc[0:50,0:5]
nyc_venues_top

NameError: name 'nyc_venues_sorted' is not defined

In [54]:
nyc_venues_top.columns

NameError: name 'nyc_venues_top' is not defined

In [55]:
m = list(nyc_venues_top['Neighbourhood'])
nyc_venues1 = nyc_venues[nyc_venues['Neighbourhood'].isin(m)]
nyc_venues1

NameError: name 'nyc_venues_top' is not defined

#### Number of categories

In [56]:

# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues1[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighbourhood'] = nyc_venues1['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

NameError: name 'nyc_venues1' is not defined

In [57]:
nyc_grouped = nyc_onehot.groupby('Neighbourhood').mean().reset_index()
nyc_grouped.head()

NameError: name 'nyc_onehot' is not defined

In [196]:
nyc_grouped.shape

(50, 361)

#### Get the top 10 for each neighbourhood

In [197]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [203]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = nyc_grouped['Neighbourhood']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Middle Eastern Restaurant,Bar,Mediterranean Restaurant,Seafood Restaurant,Greek Restaurant,Hookah Bar,Pizza Place,Café,Deli / Bodega,Bakery
1,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop,Playground,Beer Garden,Shopping Mall,Plaza,Boat or Ferry
2,Bay Ridge,Spa,Italian Restaurant,Bar,Pizza Place,Greek Restaurant,Thai Restaurant,American Restaurant,Pharmacy,Chinese Restaurant,Hookah Bar
3,Bayside,Bar,Pub,Indian Restaurant,Mexican Restaurant,Sushi Restaurant,Greek Restaurant,Pizza Place,American Restaurant,Donut Shop,Bakery
4,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Bank,Donut Shop,Grocery Store,Liquor Store,Spanish Restaurant


In [204]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5


nyc_grouped_clustering = nyc_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 1, 2, 1, 2, 1, 2, 4, 0], dtype=int32)

#### Merge the dataframe with the top 10 and the cluster for each neighbourhood

In [205]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nyc_merged = neighborhoods

# merge nyc_merged with nyc_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

nyc_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
nyc_merged[nyc_merged['Cluster Labels'].isnull()]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bronx,Fieldston,40.895437,-73.905643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Manhattan,Marble Hill,40.876551,-73.910660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Bronx,Woodlawn,40.898273,-73.867315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Bronx,Norwood,40.877224,-73.879391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Bronx,Williamsbridge,40.881039,-73.857446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Bronx,Baychester,40.866858,-73.835798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

nyc_merged_nonan = nyc_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged_nonan['Latitude'], nyc_merged_nonan['Longitude'], nyc_merged_nonan['Neighborhood'], nyc_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1 (Arts, cafe and style)

In [208]:
nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 0, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]
nyc_cluster1 = nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 0, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]
nyc_cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Carroll Gardens,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Bar,Spa,Wine Shop,Grocery Store,French Restaurant
112,Lincoln Square,Italian Restaurant,Plaza,Café,Gym / Fitness Center,Theater,Concert Hall,Performing Arts Venue,Indie Movie Theater,Coffee Shop,French Restaurant
113,Clinton,Theater,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Wine Shop,Gym,Pizza Place,Spa,Sandwich Place,American Restaurant
116,Chelsea,Coffee Shop,Art Gallery,Ice Cream Shop,Bakery,American Restaurant,Café,Park,Sandwich Place,Market,Cupcake Shop
117,Greenwich Village,Italian Restaurant,Sushi Restaurant,Café,Chinese Restaurant,Gym,Indie Movie Theater,Gourmet Shop,Seafood Restaurant,Dessert Shop,Comedy Club
121,Little Italy,Mediterranean Restaurant,Italian Restaurant,Bubble Tea Shop,Pizza Place,Ice Cream Shop,Bakery,Spa,Sandwich Place,Thai Restaurant,Chinese Restaurant
122,Soho,Italian Restaurant,Sandwich Place,Mediterranean Restaurant,Ice Cream Shop,Bakery,Coffee Shop,Seafood Restaurant,Café,Gym,Hotel
123,West Village,Italian Restaurant,Wine Bar,New American Restaurant,Bakery,Cocktail Bar,Park,American Restaurant,Pizza Place,Coffee Shop,Jazz Club
244,Chelsea,Coffee Shop,Art Gallery,Ice Cream Shop,Bakery,American Restaurant,Café,Park,Sandwich Place,Market,Cupcake Shop
248,Noho,Italian Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Grocery Store,Wine Shop,Hotel,Mexican Restaurant,Ice Cream Shop,Sushi Restaurant


#### Cluster 2 (Cosy corner)

In [209]:
nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 1, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]
nyc_cluster2 = nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 1, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]
nyc_cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,Pizza Place,Sandwich Place,Deli / Bodega,Bar,Supermarket,Mexican Restaurant,Latin American Restaurant,Bakery,Donut Shop,Fast Food Restaurant
16,Fordham,Mobile Phone Shop,Shoe Store,Pizza Place,Bank,Donut Shop,Pharmacy,Gym / Fitness Center,Supplement Shop,Bakery,Spanish Restaurant
34,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Bank,Donut Shop,Grocery Store,Liquor Store,Spanish Restaurant
46,Bay Ridge,Spa,Italian Restaurant,Bar,Pizza Place,Greek Restaurant,Thai Restaurant,American Restaurant,Pharmacy,Chinese Restaurant,Hookah Bar
64,Brooklyn Heights,Deli / Bodega,Yoga Studio,Pizza Place,Park,Gym,Bakery,Italian Restaurant,Mexican Restaurant,Juice Bar,Scenic Lookout
84,Clinton Hill,Italian Restaurant,Pizza Place,Mexican Restaurant,Wine Shop,Thai Restaurant,Yoga Studio,Japanese Restaurant,Diner,Restaurant,Indian Restaurant
101,Washington Heights,Café,Bakery,Mobile Phone Shop,Latin American Restaurant,Chinese Restaurant,Donut Shop,Pizza Place,Coffee Shop,Bank,Tapas Restaurant
130,Woodside,Grocery Store,Filipino Restaurant,Thai Restaurant,Bakery,Latin American Restaurant,Bar,Donut Shop,Pub,American Restaurant,Gourmet Shop
131,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mexican Restaurant,Thai Restaurant,Mobile Phone Shop,Spanish Restaurant,Diner,Pizza Place
274,Tudor City,Park,Café,Mexican Restaurant,Pizza Place,Deli / Bodega,Sushi Restaurant,Vietnamese Restaurant,Dog Run,Garden,Seafood Restaurant


#### Cluster 3 (Drinks and More)

In [210]:
nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 2, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Yoga Studio,Café,French Restaurant,Sushi Restaurant,New American Restaurant,Deli / Bodega
59,Prospect Heights,Bar,Mexican Restaurant,Thai Restaurant,Wine Shop,Gourmet Shop,Cocktail Bar,Café,Yoga Studio,Ice Cream Shop,Coffee Shop
62,Bushwick,Bar,Coffee Shop,Mexican Restaurant,Pizza Place,Deli / Bodega,Bakery,Thrift / Vintage Store,Discount Store,Café,Vegetarian / Vegan Restaurant
65,Cobble Hill,Coffee Shop,Playground,Pizza Place,Bar,Bakery,Italian Restaurant,Deli / Bodega,Cocktail Bar,Yoga Studio,Dive Bar
86,Downtown,Pizza Place,Burger Joint,Coffee Shop,Sandwich Place,Chinese Restaurant,Dance Studio,Performing Arts Venue,Bar,Deli / Bodega,Arts & Crafts Store
87,Boerum Hill,Coffee Shop,Dance Studio,Bar,Sandwich Place,Bakery,French Restaurant,Furniture / Home Store,Arts & Crafts Store,Yoga Studio,Kids Store
95,East Williamsburg,Bar,Deli / Bodega,Bakery,Coffee Shop,Cocktail Bar,Concert Hall,Mexican Restaurant,Music Venue,Vegetarian / Vegan Restaurant,Gym / Fitness Center
96,North Side,Coffee Shop,Pizza Place,American Restaurant,Bar,Yoga Studio,Wine Bar,Vegetarian / Vegan Restaurant,Jewelry Store,Bakery,Ice Cream Shop
97,South Side,Coffee Shop,Bar,American Restaurant,Pizza Place,Yoga Studio,Wine Bar,Breakfast Spot,Art Gallery,Mexican Restaurant,Cocktail Bar
100,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Spa,American Restaurant,Ice Cream Shop,Bar,Vietnamese Restaurant,Bubble Tea Shop,Coffee Shop


#### Cluster 4 (Annyeong-haseyo)

In [211]:
nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 3, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
115,Murray Hill,Korean Restaurant,Coffee Shop,Hotel,Sandwich Place,Pizza Place,Japanese Restaurant,Bar,Gym / Fitness Center,Burger Joint,Bakery
180,Murray Hill,Korean Restaurant,Coffee Shop,Hotel,Sandwich Place,Pizza Place,Japanese Restaurant,Bar,Gym / Fitness Center,Burger Joint,Bakery
250,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,American Restaurant,Dessert Shop,Café,Coffee Shop,Burger Joint,Hotel Bar,Gym / Fitness Center


#### Cluster 5 (Close to office)

In [212]:
nyc_merged_nonan.loc[nyc_merged_nonan['Cluster Labels'] == 4, nyc_merged_nonan.columns[[1] + list(range(5, nyc_merged_nonan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
107,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Spa,Exhibit,Juice Bar,Yoga Studio,French Restaurant,Grocery Store
109,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Sushi Restaurant,Café,Burger Joint,Gym,Gym / Fitness Center,Turkish Restaurant
114,Midtown,Coffee Shop,Hotel,Bakery,Theater,Clothing Store,Sushi Restaurant,Japanese Restaurant,Pizza Place,Cuban Restaurant,Cosmetics Shop
120,Tribeca,Park,Italian Restaurant,American Restaurant,Café,Wine Bar,Spa,Coffee Shop,Men's Store,Greek Restaurant,Hotel
127,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop,Playground,Beer Garden,Shopping Mall,Plaza,Boat or Ferry
128,Financial District,Coffee Shop,Pizza Place,American Restaurant,Cocktail Bar,Gym,Mexican Restaurant,Café,Falafel Restaurant,Event Space,Sandwich Place
139,Long Island City,Coffee Shop,Hotel,Bar,Pizza Place,Mexican Restaurant,Café,Supermarket,Gym / Fitness Center,Deli / Bodega,Post Office
247,Carnegie Hill,Coffee Shop,Pizza Place,Café,Bookstore,Gym / Fitness Center,Gym,Japanese Restaurant,Yoga Studio,Wine Shop,Pub
249,Civic Center,Coffee Shop,Hotel,Cocktail Bar,American Restaurant,French Restaurant,Spa,Park,Gym / Fitness Center,Yoga Studio,Café
271,Sutton Place,Gym / Fitness Center,Coffee Shop,Park,Italian Restaurant,Bagel Shop,Gym,Hotel,Furniture / Home Store,Health & Beauty Service,Latin American Restaurant
